## Cleaning House Price Data
This notebook outlines a comprehensive workflow for cleaning house price data from https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data. The process includes dropping columns with excessive missing values or low correlation with sale price, filling missing values with contextually appropriate replacements, and encoding categorical features to prepare the dataset for analysis and modeling. Read `data_description.txt` for a detailed description of the dataset.

In [59]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Load the Dataset
Read the raw house price data from CSV into a pandas DataFrame for cleaning and exploration.

In [60]:
df = pd.read_csv('../Data/train.csv')
df2 = pd.read_csv('../Data/test.csv')
df.head()
df2.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


### Check for Missing Values
Calculate the percentage of missing values in each column to identify which features require cleaning.

In [61]:
missing_percentage = df.isnull().mean() * 100
print(missing_percentage.sort_values(ascending=False))

PoolQC         99.520548
MiscFeature    96.301370
Alley          93.767123
Fence          80.753425
MasVnrType     59.726027
                 ...    
ExterQual       0.000000
Exterior2nd     0.000000
Exterior1st     0.000000
RoofMatl        0.000000
SalePrice       0.000000
Length: 81, dtype: float64


### Drop Columns with Too Many Missing Values
Remove columns that have a high percentage of missing values and are unlikely to be useful for modeling.

In [62]:
df.drop(columns = ['Alley', 'PoolQC', 'MiscFeature', 'Fence'], inplace=True)

### Encode Categorical Features
Convert categorical variables to numeric codes using label encoding, which is required for correlation analysis and modeling.

In [63]:
missing_percentage = df.isnull().mean() * 100
print(missing_percentage.sort_values(ascending=False))

MasVnrType     59.726027
FireplaceQu    47.260274
LotFrontage    17.739726
GarageCond      5.547945
GarageYrBlt     5.547945
                 ...    
BsmtUnfSF       0.000000
TotalBsmtSF     0.000000
MSSubClass      0.000000
HeatingQC       0.000000
SalePrice       0.000000
Length: 77, dtype: float64


### Check Feature Types and Correlations
Identify which features are categorical and which are numeric, and check their correlation with the target variable `SalePrice`.

In [64]:
features_to_check = ['MasVnrType', 'FireplaceQu', 'LotFrontage', 'GarageCond', 'GarageYrBlt']

for feature in features_to_check:
    if df[feature].dtype == 'object':
        print(f"{feature} is categorical")
    else:
        corr_value = df[feature].corr(df['SalePrice'])
        print(f"Correlation of {feature} with SalePrice: {corr_value:.3f}")


MasVnrType is categorical
FireplaceQu is categorical
Correlation of LotFrontage with SalePrice: 0.352
GarageCond is categorical
Correlation of GarageYrBlt with SalePrice: 0.486


### Encode Categorical Features
Convert categorical variables to numeric codes using label encoding, which is required for correlation analysis and modeling.

In [65]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()

# Convert MasVnrType to string first (to handle NaNs)
df_encoded['MasVnrType'] = df_encoded['MasVnrType'].astype(str)

# Label encode
le = LabelEncoder()
df_encoded['MasVnrType_encoded'] = le.fit_transform(df_encoded['MasVnrType'])

# Check correlation
corr_value = df_encoded['MasVnrType_encoded'].corr(df_encoded['SalePrice'])
print(f"Correlation of MasVnrType with SalePrice: {corr_value:.3f}")


Correlation of MasVnrType with SalePrice: -0.282


### Drop useless column
Remove `MasVnrType` column because it has too many missing values and the correlation with `SalePrice`

In [66]:
df.drop(columns=['MasVnrType'], inplace=True)
#drop column because it has too many missing values and the correlation with SalePrice is low

### Re-check Missing Values After Encoding/Dropping
Check again for missing values after encoding and dropping columns to ensure all issues are addressed before final cleaning steps.

In [67]:
missing_percentage = df.isnull().mean() * 100
print(missing_percentage.sort_values(ascending=False))

FireplaceQu     47.260274
LotFrontage     17.739726
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
                  ...    
BsmtUnfSF        0.000000
TotalBsmtSF      0.000000
Heating          0.000000
MSSubClass       0.000000
SalePrice        0.000000
Length: 76, dtype: float64


### Fill Missing Fireplace Quality
Fill missing values in the `FireplaceQu` column with 'No Fireplace' to indicate houses without a fireplace.

In [68]:
df['FireplaceQu'] = df['FireplaceQu'].fillna('No Fireplace')


### Fill Missing Garage Columns with Descriptive Value
Fill missing values in garage-related columns with the string 'No Garage' to clearly indicate the absence of a garage feature.

In [69]:
fill_garage_none = ['GarageCond', 'GarageQual', 'GarageFinish', 'GarageType', 'GarageYrBlt']
df[fill_garage_none] = df[fill_garage_none].fillna('No Garage')

missing_percentage = df.isnull().mean() * 100
print(missing_percentage.sort_values(ascending=False))


LotFrontage     17.739726
BsmtFinType2     2.602740
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtQual         2.534247
                  ...    
BsmtUnfSF        0.000000
TotalBsmtSF      0.000000
Heating          0.000000
MSSubClass       0.000000
SalePrice        0.000000
Length: 76, dtype: float64


### Fill Missing LotFrontage by Neighborhood Median
Fill missing values in `LotFrontage` with the median value for each Neighborhood, preserving local property characteristics.

In [70]:
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median())
)
df['LotFrontage'].isnull().sum()

0

### Fill Missing Basement Columns with Descriptive Value
Fill missing values in basement-related columns with the string 'No Basement' to clearly indicate the absence of a basement feature.

In [71]:
fill_basement_none = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
df[fill_basement_none] = df[fill_basement_none].fillna('No Basement')

missing_percentage = df.isnull().mean() * 100
print(missing_percentage.sort_values(ascending=False))


MasVnrArea      0.547945
Electrical      0.068493
BedroomAbvGr    0.000000
FireplaceQu     0.000000
Fireplaces      0.000000
                  ...   
ExterQual       0.000000
Exterior2nd     0.000000
Exterior1st     0.000000
RoofMatl        0.000000
SalePrice       0.000000
Length: 76, dtype: float64



### Fill Missing Values for MasVnrArea and Electrical
Fill missing values in `MasVnrArea` with 0.0 (no masonry veneer) and in `Electrical` with the mode (most common value).

In [72]:
df['MasVnrArea'] = df['MasVnrArea'].fillna(0.0)
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

missing_percentage = df.isnull().mean() * 100
print(missing_percentage.sort_values(ascending=False))

Id             0.0
HalfBath       0.0
FireplaceQu    0.0
Fireplaces     0.0
Functional     0.0
              ... 
MasVnrArea     0.0
Exterior2nd    0.0
Exterior1st    0.0
RoofMatl       0.0
SalePrice      0.0
Length: 76, dtype: float64


### Preview the Cleaned Data
Display the first few rows of the cleaned DataFrame to verify the results of the cleaning process.

In [73]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


### Save the Cleaned Data
Export the cleaned DataFrame to a new CSV file for use in further analysis and modeling.

In [74]:
save_path = '../Data/cleaned_train.csv'
df.to_csv(save_path, index=False)

### Cleaning Test Data
Cleaning the test data follows a similar process to the training data. We will drop columns with excessive missing values, fill missing values with appropriate replacements, and encode categorical features.

In [75]:
# Drop columns with too many missing values
df2.drop(columns=['Alley', 'PoolQC', 'MiscFeature', 'Fence'], inplace=True)
# Drop MasVnrType if present
if 'MasVnrType' in df2.columns:
    df2.drop(columns=['MasVnrType'], inplace=True)
# Fill missing Fireplace Quality
if 'FireplaceQu' in df2.columns:
    df2['FireplaceQu'] = df2['FireplaceQu'].fillna('No Fireplace')
# Fill missing Garage columns
fill_garage_none = ['GarageCond', 'GarageQual', 'GarageFinish', 'GarageType', 'GarageYrBlt']
for col in fill_garage_none:
    if col in df2.columns:
        df2[col] = df2[col].fillna('No Garage')
# Fill missing LotFrontage by Neighborhood median
if 'LotFrontage' in df2.columns and 'Neighborhood' in df2.columns:
    df2['LotFrontage'] = df2.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
# Fill missing Basement columns
fill_basement_none = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
for col in fill_basement_none:
    if col in df2.columns:
        df2[col] = df2[col].fillna('No Basement')
# Fill MasVnrArea and Electrical
if 'MasVnrArea' in df2.columns:
    df2['MasVnrArea'] = df2['MasVnrArea'].fillna(0.0)
if 'Electrical' in df2.columns:
    df2['Electrical'] = df2['Electrical'].fillna(df2['Electrical'].mode()[0])

In [77]:
# Fill specific missing values in df2 as required for submission
df2['MSZoning'] = df2['MSZoning'].fillna(df2['MSZoning'].mode()[0])
df2['Utilities'] = df2['Utilities'].fillna(df2['Utilities'].mode()[0])
df2['Exterior1st'] = df2['Exterior1st'].fillna(df2['Exterior1st'].mode()[0])
df2['Exterior2nd'] = df2['Exterior2nd'].fillna(df2['Exterior2nd'].mode()[0])
df2['BsmtFinSF1'] = df2['BsmtFinSF1'].fillna(0)
df2['BsmtFinSF2'] = df2['BsmtFinSF2'].fillna(0)
df2['BsmtUnfSF'] = df2['BsmtUnfSF'].fillna(0)
df2['TotalBsmtSF'] = df2['TotalBsmtSF'].fillna(0)
df2['BsmtFullBath'] = df2['BsmtFullBath'].fillna(0)
df2['BsmtHalfBath'] = df2['BsmtHalfBath'].fillna(0)
df2['KitchenQual'] = df2['KitchenQual'].fillna(df2['KitchenQual'].mode()[0])
df2['Functional'] = df2['Functional'].fillna(df2['Functional'].mode()[0])
df2['GarageCars'] = df2['GarageCars'].fillna(0)
df2['GarageArea'] = df2['GarageArea'].fillna(0)
df2['SaleType'] = df2['SaleType'].fillna(df2['SaleType'].mode()[0])
# Save cleaned test set
df2.to_csv('../Data/cleaned_test.csv', index=False)